# Selenium

In [1]:
# @title Importing Libraries

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from config import *
import time
driver = webdriver.Chrome()


## Driver startup
The following code will go to twitter and accept the cookies

In [2]:
#get twitter.com
driver.get("https://twitter.com/") #driver.get returns after the page is loaded but just to be sure, I'll wait 5 more seconds
print("Waiting for 5 seconds")
time.sleep(5)
print("Done Waiting")
#click on the full xpath to accept cookies
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div/div[2]/div[1]').click()


Waiting for 5 seconds
Done Waiting


In [3]:
#capture all the elements whose tag is 'article'
def getTweets(tagName = 'article'):
    tweets = driver.find_elements(By.TAG_NAME, tagName)
    return tweets

In [4]:
def getTweetContent(tweet):
    return " ".join(tweet.text.split('\n')[4:-4])

In [5]:
def printCurrentTweets(tweets = []):
    """
    This function prints the current tweets on the page that can be located through the tag name 'article'
    """
    tweets = tweets if len(tweets) > 0 else getTweets()
    for tweet in tweets:
        body = getTweetContent(tweet)
        print(body)
        print("--------------------------------------------------")
        
        

    

In [6]:
def colorBackground(tweet, color = "red"):
    
    driver.execute_script("arguments[0].style.backgroundColor = '"+color+"';", tweet)

In [7]:
tweets = getTweets()

In [8]:
printCurrentTweets()

Il giovane più splendente della stagione  Nicolo Fagioli è il miglior U23 della stagione di #SerieATIM   @EA_FIFA_Italia
--------------------------------------------------
 Moise Kean a http://views.fr: "C'è una storia che non dimenticherò mai. Avevo 15 anni, entrai per la prima volta nello spogliatoio della Juventus, e lì Manzukic mi disse 'Cosa ci fai qui'. Poi continua 'Non pensare di restare qui, rimani forse 1 settimana e poi te ne…
--------------------------------------------------
La tentata aggressione con insulti all'arbitro di Roma-Siviglia è (anche) diretta conseguenza dello show del solito Mourinho nel garage dello stadio dopo la partita. Spero sia chiaro a tutti.
--------------------------------------------------
Non ironicamente la Juve migliore degli ultimi tre anni
--------------------------------------------------
Stronger Together.  Il 7 giugno via alle vendite degli abbonamenti  juventus.com STRONGER TOGETHER: IL 7 GIUGNO VIA ALLA VENDITA DEGLI  ABBONAMENTI! 531
----

# Neural Network

In [9]:
import tensorflow  as  tf
from  tensorflow  import  keras

from tensorflow.keras import layers
from keras.utils import pad_sequences


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sklearn

import numpy as  np
import matplotlib.pyplot  as  plt
import os

import random

import pickle

import pandas  as  pd
import tqdm as tqdm

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


#import  the confusion matrix
from sklearn.metrics import confusion_matrix


from keras.preprocessing import sequence


## Load Models

In [10]:
#load the ./disasterModel.h5 model using keras
model = keras.models.load_model('./models/disasterModel.h5', compile=False)
#compile the model
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
#load the tokenizer using pickle
with open('./models/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

## Preprocessing

In [11]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hazem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Text Cleanup

This function takes a raw text string and applies a standard NLP preprocessing pipeline consisting of the following steps:
- Text cleaning
- Tokenization
- Stopwords removal
- Stemming (Snowball stemmer)


In [12]:
def clean_text(text):
    
    # Text cleaning pipeline
    
    # 1. Text cleaning
    # 1.a Case normalization
    text = text.lower()
    
    # 1.b Trimming
    text = text.strip()
    
    # 1.c Filter out punctuation symbols
    text = re.sub("[^a-zA-Z\s]", "", text)
    
    # 1.d Filter out any internal extra whitespace
    text = re.sub(" +", " ", text)
    
    # 2. Tokenization (split text into tokens)
    tokens = text.split()
    
    # 3. Stopwords removal
    stop_words = set(stopwords.words("english"))
    terms = [token for token in tokens if token not in stop_words]
    
    # 4. Stemming (Snowball stemmer)
    stemmer = SnowballStemmer(language="english")
    terms_stemmed = [stemmer.stem(term) for term in terms]
    
    # Return the cleaned text
    return " ".join(terms_stemmed)


In [13]:
def prepareText(rawTweet):
    """
    This function takes a raw tweet and returns a list of cleaned tokens
    """
    #clean the raw tweet
    cleanTweet = clean_text(getTweetContent(rawTweet))
    #tokenize the clean tweet
    tokens = tokenizer.texts_to_sequences([cleanTweet])
    #pad the tokens
    paddedTokens = pad_sequences(tokens, maxlen=23, padding='post')
    return paddedTokens

## Function to change the tweet background

In [14]:
def predictLive():
    tweets = getTweets()
    for tweet in tweets:
        prediction = np.round(model.predict(prepareText(tweet))[0]) #1 disaster, 0 not disaster
        if prediction == 1:
            colorBackground(tweet, "red")
        else:
            colorBackground(tweet, "green")
    

## Input Handling

Now we need a way to let the user interact with the predictor. 
Whenever the user presses ENTER, the application will fetch the tweets nearby and will classify them accordingly

In [15]:
from pynput import keyboard

# Define the code to be executed when the key is pressed
def handle_key_press(key):
    if key == keyboard.Key.enter:
        print("Enter key pressed!")
        #prevent the user from scrolling down
        try:
            predictLive()
        except:
            print("Couldn't find any tweets")


    elif key == keyboard.Key.esc:
        print("Escape key pressed!")
        print("Exiting...")
        # Stop listener
        return False
    elif key == keyboard.Key.shift_r:
        #take a screenshot
        #get time stamp 
        timestamp = time.strftime("%Y%m%d-%H%M%S")

        driver.save_screenshot(f"./screenshots/{timestamp}.png")
        print("Screenshot taken!")
    

# Create a listener for key press events
listener = keyboard.Listener(on_press=handle_key_press)

# Start the listener
listener.start()

# Keep the program running until a specific key is pressed (e.g., Escape)
listener.join()


Enter key pressed!
1/1 [==============================] - 0s 16ms/step
Enter key pressed!
1/1 [==============================] - 0s 26ms/step
Enter key pressed!
1/1 [==============================] - 0s 16ms/step
Enter key pressed!
1/1 [==============================] - 0s 24ms/step
Enter key pressed!
1/1 [==============================] - 0s 21ms/step
Enter key pressed!
1/1 [==============================] - 0s 15ms/step
Enter key pressed!
1/1 [==============================] - 0s 14ms/step
Enter key pressed!
1/1 [==============================] - 0s 14ms/step
Couldn't find any tweets
Enter key pressed!
1/1 [==============================] - 0s 18ms/step
Enter key pressed!
Enter key pressed!
1/1 [==============================] - 0s 17ms/step
Enter key pressed!
1/1 [==============================] - 0s 16ms/step
Enter key pressed!
1/1 [==============================] - 0s 14ms/step
Enter key pressed!
1/1 [==============================] - 0s 21ms/step
Enter key pressed!
1/1 [=========